In [1]:
song_url = "https://www.hooktheory.com/theorytab/view/jay-hardway/stardust"
# song_url = "./Data/Stardust by Jay Hardway Chords, Melody, and Music Theory Analysis - Hooktheory.html"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import datetime
import os

from selenium import webdriver
from bs4 import BeautifulSoup
import requests

## Microsoft Edge Remote Debugging Port (tránh bị nhận diện phần mềm automation)

In [3]:
browser_instance_data_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/browser_instance_data"
port_to_run_browser = 40000

command = f'''open -na "Microsoft Edge.app" --args
            --remote-debugging-port={port_to_run_browser}
            --user-data-dir="{browser_instance_data_dir}"'''

os.system(command.replace("\n", " "))

0

## Kết nối Selenium đến trình duyệt Edge ở port được mở

In [4]:
options = webdriver.EdgeOptions()
options.add_experimental_option("debuggerAddress", f"localhost:{port_to_run_browser}")
# options.add_argument("headless") # Chạy Selenium ở chế độ headless (không hiển thị giao diện browser)

browser = webdriver.Edge(options=options)
browser.maximize_window() # Phóng to cửa sổ trình duyệt

In [5]:
import time

# Lướt xuống cuối trang với nhiều lần scroll cách nhau 1 giây
def scroll_to_bottom(webdriver_instance=browser):
    scroll_pause_time = 1
    screen_height = browser.execute_script("return window.screen.height;") # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        browser.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = browser.execute_script("return document.body.scrollHeight;")
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break

In [6]:
# Find all <svg> tags and horizontaly scroll each of them to the right
def scroll_svg_to_right(webdriver_instance=browser):
    # Tìm các thẻ svg
    # svgs = browser.find_elements(By.TAG_NAME, "svg")
    pass
    # svgs = browser.execute_script('''return document.querySelectorAll('svg[data-type=\"score\"]')''')

    # # Lần lượt scroll từng thẻ svg sang phải
    # for svg in svgs:
    #     # Lấy kích thước của thẻ svg
    #     svg_size = svg.size
    #     # Lấy tọa độ x của thẻ svg
    #     svg_x = svg.location["x"]
    #     # Lấy tọa độ y của thẻ svg
    #     svg_y = svg.location["y"]
    #     # Scroll thẻ svg sang phải
    #     browser.execute_script(f"window.scrollTo({svg_x + svg_size['width']}, {svg_y});")

In [7]:
def get_html_content(url, webdriver_instance=browser):
    # Mở trang web
    webdriver_instance.get(url)

    # Lướt từ đầu đến cuối trang web
    scroll_to_bottom(webdriver_instance)
    # scroll_svg_to_right(webdriver_instance)
    
    # Lấy mã nguồn HTML của trang
    contents = webdriver_instance.page_source

    return contents

In [8]:
def get_specific_part_of_a_song(html_content, part_of_the_song):
    """
    Lấy thông tin đoạn được chỉ định của bài hát cho trước
    :param html_content: nội dung html của trang web chứa bài hát
    :param part_of_the_song: đoạn cần lấy. Ví dụ: "Intro and Verse", "Chorus", v.v.
    :return: thẻ div chứa phần của bài hát cần tìm
    """

    # Lấy thông tin html của trang web 
    # response = requests.get(url)
    # contents = response.text

    


    # # Mở file HTML và đọc nội dung
    # with open(url, 'r') as f:
    #     contents = f.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')

    # # Tìm tất cả thẻ div
    # divs = soup.find_all('div')

    # # Duyệt qua từng thẻ div
    # for i in range(len(divs)):
    #     # Kiểm tra xem thẻ div có chứa thẻ h2 với văn bản = part_of_the_song không (với điều kiện thẻ h2 đó nằm ở cấp thứ 4 trong thẻ div)
    #     if divs[i].find('h2', string=part_of_the_song):
    #         # Nếu có, trả về thẻ div ngang cấp tiếp theo
    #         if i+1 < len(divs):
    #             return divs[i+1]
    #             # Trả về thẻ g có data type là line
    #             return divs[i+1].find('g', {'data-type': 'line'})
            
    # # Duyệt qua từng thẻ div
    # for div in divs:
    #     # Kiểm tra xem thẻ div có chứa thẻ h2 với string là "Chorus" không
    #     h2 = div.find('h2', string='Chorus')

    #     # Kiểm tra xem thẻ div có chứa thẻ div khác chứa thẻ g dạng "segment" không
    #     segment_g = div.find('g', {'data-type': 'segment'})

    #     # Nếu thỏa mãn cả hai điều kiện, trả về thẻ div
    #     if h2 and segment_g:
    #         return div

    # Tìm thẻ h2
    h2 = soup.find('h2', string='Chorus')

    # print(h2)
    # print(h2.parent.parent.parent.find_next_sibling("div"))
    
    # if h2 and h2.parent and h2.parent.parent and h2.parent.parent.parent and h2.parent.parent.parent.name == 'div':
    if h2 and h2.parent.parent.parent.name == 'div':
        # next_sibling = h2.parent.parent.parent.next_sibling
        # Lấy thẻ svg có data-type là score
        # if next_sibling and next_sibling.name == 'div':
        svg = h2.parent.parent.parent.find_next_sibling("div").find('svg', {'data-type': 'score'})
        # svg = h2.parent.parent.parent.find_next_sibling("div").find('div', {'id': 'tab-281819hookpad-score-div'})
        return svg
    return None

In [9]:
html_content = get_html_content(song_url, browser)
print(html_content)

# Save to file
def save_to_file(html_content, file_path):
    with open(file_path, "w") as f:
        f.write(html_content)

save_to_file(html_content, "./html/Stardust by Jay Hardway Chords, Melody, and Music Theory Analysis - Hooktheory.html")

<html lang="en" class="hydrated"><head><style data-emotion="css" data-s=""></style><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style>
  <meta name="description" content="Chords, melody, and music theory analysis of Stardust by Jay Hardway.">
<link rel="stylesheet" type="text/css" href="/lib/bootstrap/css/bootstrap.min.css">
<link rel="stylesheet" type="text/css" href="/lib/fontello-1b936d28/css/animation.css">
<link rel="stylesheet" type="text/css" href="/lib/fontello-1b936d28/css/fontello-codes.css">
<link rel="stylesheet" type="text/css" href="/lib/fontello-1b936d28/css/fontello.css">
<link rel="stylesheet" type="text/css" href="/build/css/style.min.css?ts=20231107163750">
<script type="text/javascript" id="www-widgetapi-script" src="https://www.youtube.com/s/player/787

In [10]:
# Chạy thử get_next_div với url của bài hát
chorus_svg = get_specific_part_of_a_song(
    get_html_content(song_url, browser), 
    "Chorus"
)

# Kiểm tra kết quả
print(chorus_svg.prettify())

# Lưu nội dung html vào file
save_to_file(chorus_svg.prettify(), "./html/Stardust by Jay Hardway Chords, Melody, and Music Theory Analysis - Hooktheory - Chorus.html")


<svg data-type="score" height="229" overflow="visible" viewbox="0 0 1451 229" width="1451">
 <rect fill="rgb(255 255 255)" height="229" opacity="1" width="1451" x="0" y="0">
  <title>
  </title>
 </rect>
 <defs>
  <pattern height="10" id="pattern-raw-1" patterntransform="rotate(45)" patternunits="userSpaceOnUse" width="10">
   <rect fill="#ff0000" height="10" width="5" x="0" y="0">
    <title>
    </title>
   </rect>
   <rect fill="#ffb014" height="10" width="5" x="5" y="0">
    <title>
    </title>
   </rect>
  </pattern>
  <pattern height="10" id="pattern-raw-3" patterntransform="rotate(45)" patternunits="userSpaceOnUse" width="10">
   <rect fill="#ffb014" height="10" width="5" x="0" y="0">
    <title>
    </title>
   </rect>
   <rect fill="#EFE600" height="10" width="5" x="5" y="0">
    <title>
    </title>
   </rect>
  </pattern>
  <pattern height="10" id="pattern-raw-6" patterntransform="rotate(45)" patternunits="userSpaceOnUse" width="10">
   <rect fill="#00D300" height="10" widt

In [11]:
os.system("pwd")

/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling


0

In [50]:
import pandas as pd

# Lấy thẻ g có data-type đúng bằng note-staff
note_staff_g = chorus_svg.find('g', {'data-type': 'note-staff '})

# Xoá các thẻ g trong note_staff_g
for g in note_staff_g.find_all('g'):
    g.decompose()

# Prettyfy thẻ g in ra màn hình và lưu vào file
save_to_file(note_staff_g.prettify(), "./html/Chorus - note-staff.html")

# Lấy tất cả thẻ rect trừ thẻ rect đầu tiên
rects = note_staff_g.find_all('rect')[1:]

# Đưa thông tin các thẻ rect vào dataframe với các cột lần lượt là các thuộc tính x, y, width, height, opacity
df = pd.DataFrame(
    data=[[rect["x"], rect["y"], rect["width"], rect["height"], rect["opacity"]] for rect in rects],
    columns=["x", "y", "width", "height", "opacity"]
)

# In dataframe
df

,x,y,width,height,opacity
0,36,133,9.046875,10,1
1,47.046875,122,9.046875,10,1
2,58.09375,122,9.046875,10,1
3,69.140625,122,9.046875,10,0
4,80.1875,100,9.046875,10,1
5,91.234375,122,9.046875,10,0
6,102.28125,111,9.046875,10,1
7,113.328125,122,9.046875,10,1
8,124.375,133,9.046875,10,1
9,135.421875,122,9.046875,10,1


In [13]:
# browser.quit()
# os.system(f"lsof -ti tcp:{port_to_run_browser} | xargs kill -9")